In [7]:
#import stuff
import pickle
import numpy as np
import pandas as pd
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import check_pairwise_arrays
from sklearn.preprocessing import normalize

%run utils.ipynb

This is an adaption of the basic VSM model. It resembles part of an old version of the file which is used as a resource file for the TieredIndex as it doesn't work correctly with the latest version of the basic VSM.

In [8]:
#get the query vector
def createQueryVectorTI(query, inverted_index):
    '''
    creates the query vector.
    
    query: list of unique terms of the query
    inverted_index: the regular inverted index
    '''
    rtn = np.zeros(len(inverted_index))
    for term in query:
        #print(term)
        try: 
            pos = list(inverted_index.keys()).index(term)
            rtn[pos] = 1
        except ValueError:
            pass
    if (np.sum(rtn)==0):
        raise ValueError('None of the search terms is part of the collection.')
    return rtn

In [9]:
#procedure that takes one doc and outputs the corresponding vector.
def createVectorTI(d,inverted_index,collection,idf): #d is the index of a document
    #look up, bag of words of d
    doc = collection[d]
    #compute term frequency
    f = np.unique(doc,return_counts=True)
    max_f = np.max(f[1])
    tf = 1+np.log(f[1])
    tf = tf/(1 + np.log(max_f))
    #create vector
    rtn = np.zeros(len(inverted_index))
    for i in range(len(f[0])): # for loop with ints running through the bag of words of doc
        term = f[0][i] # the term at position i
        #find correct position in vector
        if term not in inverted_index:
            continue
        pos = list(inverted_index.keys()).index(term)
        #insert tf-idf of term
        rtn[pos] = idf[term] * tf[i]
    return rtn